In [1]:
import requests
import pandas as pd
from pydantic import BaseModel, Field, RootModel
from datetime import datetime

try:
    youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    res = requests.get(youbike_url)
    res.raise_for_status()
except Exception as e:
    print(e)
else:
    print('下載成功')


下載成功


In [2]:
class Site(BaseModel):
    區域: str = Field(alias='sarea')
    總數: int = Field(alias='total')
    可借: int = Field(alias='available_rent_bikes')
    可還: int = Field(alias='available_return_bikes')
    時間: datetime = Field(alias='mday')

class YouBike(RootModel):
    root: list[Site]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

In [3]:
df_sites = pd.DataFrame(all_sites)
df_sites

,區域,總數,可借,可還,時間
0,大安區,28,6,22,2024-07-01 19:53:21
1,大安區,21,13,8,2024-07-01 19:50:33
2,大安區,16,7,9,2024-07-01 19:45:21
3,大安區,11,1,10,2024-07-01 19:54:20
4,大安區,16,0,16,2024-07-01 19:47:15
...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 19:47:15
1425,臺大公館校區,20,0,20,2024-07-01 19:53:21
1426,臺大公館校區,24,5,18,2024-07-01 19:52:19
1427,臺大公館校區,40,30,1,2024-07-01 19:54:20


In [8]:
df_sites = pd.DataFrame(all_sites)
df = df_sites.groupby(by='區域')[['總數', '可借', '可還']].agg([('車輛數', 'sum'), ('站點數', 'count')])
df

總數         可借         可還     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
區域                                     
中山區     3819  156  1570  156  2223  156
中正區     3209  121  1499  121  1683  121
信義區     3331  106  1290  106  1939  106
內湖區     3499  168  1439  168  2022  168
北投區     2472   94   933   94  1509   94
南港區     2317   92  1100   92  1196   92
士林區     2937  127  1056  127  1810  127
大同區     1629   62   603   62  1012   62
大安區     4987  185  2146  185  2752  185
文山區     2203   97   842   97  1358   97
松山區     2343   88   894   88  1429   88
臺大公館校區  1437   53   528   53   874   53
萬華區     2057   80  1005   80  1031   80

In [11]:
# pivot
# stack, unstack
# 欄位有名稱
s1 = df.stack(level=[0, 1])
s1.index.names = ['區域', '借還', '數量']
s1.unstack(level=['數量'])

C:\Users\chesterXalan\AppData\Local\Temp\ipykernel_17604\3242222421.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s1 = df.stack(level=[0, 1])


數量          車輛數  站點數
區域     借還           
中山區    可借  1570  156
       可還  2223  156
       總數  3819  156
中正區    可借  1499  121
       可還  1683  121
       總數  3209  121
信義區    可借  1290  106
       可還  1939  106
       總數  3331  106
內湖區    可借  1439  168
       可還  2022  168
       總數  3499  168
北投區    可借   933   94
       可還  1509   94
       總數  2472   94
南港區    可借  1100   92
       可還  1196   92
       總數  2317   92
士林區    可借  1056  127
       可還  1810  127
       總數  2937  127
大同區    可借   603   62
       可還  1012   62
       總數  1629   62
大安區    可借  2146  185
       可還  2752  185
       總數  4987  185
文山區    可借   842   97
       可還  1358   97
       總數  2203   97
松山區    可借   894   88
       可還  1429   88
       總數  2343   88
臺大公館校區 可借   528   53
       可還   874   53
       總數  1437   53
萬華區    可借  1005   80
       可還  1031   80
       總數  2057   80